# Requirements

In [ ]:
import subprocess
reqs = open("requirements.txt", "r").read().split('\n')
print("Installing: ", reqs)
subprocess.run(["py", "-m", "pip", "install", "uv"])
for x in reqs: 
    subprocess.run(["py", "-m", "uv", "pip", "install", x])

# Imports

In [ ]:
import pyspark.sql.functions as spark_func
from pyspark.sql import SparkSession, Row, DataFrame
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder

import json
import pprint
import numpy as np
from IPython.core.display import HTML
from IPython.display import display

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

# Spark Session

In [ ]:
spark = SparkSession.builder.appName("EDA").getOrCreate()
spark

# Create dataframes

In [ ]:
train_df = spark.read.option("header", True).csv("resources/fraudTrain.csv")
display(HTML("<style>pre { white-space: pre !important; }</style>"))
train_df.show()

In [ ]:
test_df = spark.read.option("header", True).csv("resources/fraudTest.csv")
test_df.show()

# EDA on training data

## Columns
- index - Unique Identifier for each row

**Trasactions**
- trans_date_trans_time - Transaction DateTime
- amt - Amount of Transaction
- trans_num - Transaction Number
- unix_time - UNIX Time of transaction

**Personal**
- first - First Name of Credit Card Holder
- last - Last Name of Credit Card Holder
- gender - Gender of Credit Card Holder
- cc_num - Credit Card Number of Customer
- job - Job of Credit Card Holder
- dob - Date of Birth of Credit Card Holder

**Adress**
- street - Street Address of Credit Card Holder
- city - City of Credit Card Holder
- state - State of Credit Card Holder
- zip - Zip of Credit Card Holder
- lat - Latitude Location of Credit Card Holder
- long - Longitude Location of Credit Card Holder
- city_pop - Credit Card Holder's City Population

**Merchant**
- merchant - Merchant Name
- category - Category of Merchant
- merch_lat - Latitude Location of Merchant
- merch_long - Longitude Location of Merchant

**Target**
- is_fraud - Fraud Flag <--- Target Class

## New features
**Transactions**
- year, month, day, hour, miute (datetime)
- amt_even (power = 0.25)
- amt diff between mean merchant amt
- amt diff between mean category amt
- amt diff between mean gender amt
- amt diff between mean city amt
- amt diff between mean street amt

**Merchant**
- merchant levels (risks 0, 1, 2)
- category one hot encoding
- category high risk
- historical merchant fraud
- historicla category fraud

**Personal**
- gender one hot
- city_pop

## Target feature `is_fraud`

In [ ]:
train_df = train_df.withColumn("is_fraud", spark_func.col("is_fraud").cast("int")) 
non_fraud_records = train_df.where(spark_func.col("is_fraud") == 0).count()
fraud_records = train_df.where(spark_func.col("is_fraud") == 1).count()
print(f"Non-fraud = {non_fraud_records}\nFraud = {fraud_records}\nFraud ratio = {round(fraud_records / train_df.count(), 3)}")

## Transaction date
- histograms do not show any outliers
- significant increase in fraud ratio in night hours (from 10 p.m to 3 a.m.)

In [ ]:
def add_date_features(dataframe):
    dataframe = dataframe.withColumn(
        "date",
        spark_func.to_date(dataframe["trans_date_trans_time"], "yyyy-MM-dd HH:mm:ss")
    )
    dataframe = (
        dataframe
        .withColumn("year", spark_func.year(dataframe["date"]))
        .withColumn("month", spark_func.month(dataframe["date"]))
        .withColumn("day", spark_func.day(dataframe["date"]))
        .withColumn("hour", spark_func.day(dataframe["date"]))
    )
    dataframe = dataframe.withColumn(
        "hour",
        spark_func.hour(
            spark_func.to_timestamp(dataframe["trans_date_trans_time"], "yyyy-MM-dd HH:mm:ss")
        )
    )
    dataframe = dataframe.withColumn(
        "minute",
        spark_func.minute(
            spark_func.to_timestamp(dataframe["trans_date_trans_time"], "yyyy-MM-dd HH:mm:ss")
        )
    )
    return dataframe

In [ ]:
train_df = add_date_features(train_df)

In [ ]:
graph_data = train_df.select("date", "year", "month", "day", "hour", "minute", "is_fraud").toPandas()
graph_data["is_fraud"] = graph_data["is_fraud"].astype(int)


for i, col in enumerate(["year", "month", "day", "hour", "minute"]):
    current_df = graph_data.groupby(col).agg({"is_fraud": "mean"}).reset_index()
    current_df["is_fraud_text"] = current_df["is_fraud"].apply(lambda x: str(round(x*100, 2)) + "%")

    fig = make_subplots(
        cols=1, 
        rows=2,
        subplot_titles=("Histogram", "Fraud ratio")
    )
    fig.add_trace(
        go.Histogram(
            x=graph_data[col],
            xbins=dict(size=1)
        ),
        col=1,
        row=1
    )
    fig.add_trace(
        go.Bar(
            x=current_df[col].to_list(),
            y=(current_df["is_fraud"]).to_list(),
            text=current_df["is_fraud_text"]
        ),
        col=1,
        row=2
    )
    fig.update_layout(
        title=f"<b>Feature: {col}</b>",
        height=600,
        width=1000,
    )
    fig.show()

## Merchant & merchant category

In [ ]:
m = train_df.select("merchant").distinct().count()
c = train_df.select("category").distinct().count()
total = train_df.count()
print(f"Toal train rows = {total}\nMerchant unique values: {m}\nMerchant category unique values = {c}")

In [ ]:
graph_data = train_df.select("merchant", "category", "is_fraud").toPandas()
graph_data["is_fraud"] = graph_data["is_fraud"].astype(int)
graph_data

### Merchant
- `meracht_lvl_0 / 1 / 2` - a category of merchat due to the fraud frquency
- `merchant_fraud_history` - hisotrical fraud ratio

In [ ]:
current_df = graph_data.groupby("merchant").agg({"is_fraud": "mean"}).reset_index().sort_values("is_fraud", ascending=False)

In [ ]:
px.bar(
    current_df,
    x="merchant",
    y="is_fraud",
    width=1000
).show()

In [ ]:
def add_merchant_level(dataframe, lvl0_values: list[str], lvl1_values: list[str], lvl2_values: list[str]):
    dataframe = (
        dataframe
        .withColumn("merchant_lvl_0", spark_func.when(spark_func.col("merchant").isin(lvl0_values), 1).otherwise(0))
        .withColumn("merchant_lvl_1", spark_func.when(spark_func.col("merchant").isin(lvl1_values), 1).otherwise(0))
        .withColumn("merchant_lvl_2", spark_func.when(spark_func.col("merchant").isin(lvl2_values), 1).otherwise(0))
    )
    return dataframe

In [ ]:
q33 = current_df["is_fraud"].quantile(0.33)
q66 = current_df["is_fraud"].quantile(0.33)

lvl0_values = current_df[current_df["is_fraud"] < q33]["merchant"].to_list()
lvl1_values = current_df[(current_df["is_fraud"] >= q33) & (current_df["is_fraud"] < q66)]["merchant"].to_list()
lvl2_values = current_df[current_df["is_fraud"] >= q66]["merchant"].to_list()

json.dump(lvl0_values, open("resources/lvl0_merchants.json", "w"), indent=4)
json.dump(lvl1_values, open("resources/lvl1_merchants.json", "w"), indent=4)
json.dump(lvl2_values, open("resources/lvl2_merchants.json", "w"), indent=4)

In [ ]:
q33 = current_df["is_fraud"].quantile(0.33)
q66 = current_df["is_fraud"].quantile(0.33)
train_df = add_merchant_level(
    train_df,
    lvl0_values=json.load(open("resources/lvl0_merchants.json", "r")), 
    lvl1_values=json.load(open("resources/lvl1_merchants.json", "r")),
    lvl2_values=json.load(open("resources/lvl2_merchants.json", "r"))
)
train_df.select("merchant_lvl_0", "merchant_lvl_1", "merchant_lvl_2").show()

In [ ]:
n = int(train_df.count() * 0.2)
random_rows = train_df.orderBy(spark_func.rand()).limit(n)


df_fraud_history = random_rows.groupby("merchant").agg(spark_func.sum("is_fraud").alias("merchant_fraud_history"))
train_df = train_df.join(df_fraud_history, on="merchant", how="right")
train_df.count()

In [ ]:
df_fraud_history.toPandas().to_csv("resources/merchant_fraud_history.csv", sep=",")

### Category merchant
- significant caterogy: caegories with high fraud ratio
- one hot encoding for category
- historical fraud ratio for categories

In [ ]:
current_df = graph_data.groupby("category").agg({"is_fraud": "mean"}).reset_index().sort_values("is_fraud", ascending=False)

In [ ]:
px.bar(
    current_df,
    x="category",
    y="is_fraud",
    width=1000
).show()

In [ ]:
px.histogram(
    graph_data,
    x="category",
    width=1000
).show()

In [ ]:
def sig_category(dataframe, categories: list[str]):
    return (
        dataframe
        .withColumn(
            "sig_category",
            spark_func.when(spark_func.col("category").isin(categories), 1).otherwise(0)
        )
    )

In [ ]:
def category_dummy(dataframe):
    indexer = StringIndexer(inputCol="category", outputCol="category_index")
    dataframe = indexer.fit(dataframe).transform(dataframe)
    
    encoder = OneHotEncoder(inputCol="category_index", outputCol="category_dummy")
    dataframe = encoder.fit(dataframe).transform(dataframe)
    
    return dataframe

In [ ]:
train_df = sig_category(train_df, ["grocery_pos", "misc_net", "shopping_net"])
train_df = category_dummy(train_df)
train_df.select("sig_category", "category_dummy").show(truncate=False)

In [ ]:
n = int(train_df.count() * 0.2)
random_rows = train_df.orderBy(spark_func.rand()).limit(n)


df_fraud_history = random_rows.groupby("category").agg(spark_func.sum("is_fraud").alias("category_fraud_history"))
train_df = train_df.join(df_fraud_history, on="category", how="right")
train_df.count()

In [ ]:
df_fraud_history.toPandas().to_csv("resources/categories_fraud_history.csv", sep=",")

## Transaction amount

**Non-fraud transactions**
- strong positive skewness
- many records in range 40 - 70
- lower median (~48)
- lots of extremely high outliers

**Frad transactions**
- weak positive skewness
- higher median (~400)
- more even distribution

### Features
- transformed amt (power = 0.25)
- transacion diff between current vlue and mean merchant value
- transacion diff between current vlue and mean category value

In [ ]:
def show_histograms(dataframe, column: str, p = None):
    hist_data = dataframe.select(column, "is_fraud").toPandas()
    hist_data[column] = hist_data[column].astype(float)
    hist_data["is_fraud"] = hist_data["is_fraud"].astype(int)

    if p:
        hist_data[column] = np.power(hist_data[column], p)
    
    px.histogram(
        hist_data[hist_data["is_fraud"] == 0],
        x=column,
        width=1000,
        title=f"<b>Fraud = False</b><br>Median = {hist_data[hist_data['is_fraud'] == 0][column].median()}"
    ).show()
    px.histogram(
        hist_data[hist_data["is_fraud"] == 1],
        x=column,
        width=1000,
        title=f"<b>Fraud = True</b><br>Median = {hist_data[hist_data['is_fraud'] == 1][column].median()}"
    ).show()

In [ ]:
show_histograms(train_df, "amt")

### Transform distribution with power < 1

In [ ]:
show_histograms(train_df, "amt", 0.25)

In [ ]:
def transform_amt(dataframe, power):
    dataframe = dataframe.withColumn("amt", spark_func.col("amt").cast("float"))
    dataframe = dataframe.withColumn("amt_even", spark_func.power(dataframe["amt"], power))

    # diff between categorical features
    for col in ["merchant", "category", "gender", "state", "city"]:
        dataframe = dataframe.join(
            dataframe.groupBy("merchant").agg(spark_func.mean("amt_even").alias(f"mean_{col}_amt")), 
            on="merchant", 
            how="right"
        )
        dataframe = dataframe.withColumn(f"amt_{col}_diff", spark_func.col("amt_even") - spark_func.col(f"mean_{col}_amt"))

    return dataframe

In [ ]:
train_df = transform_amt(train_df, 0.25)
train_df.select(
    "amt", "amt_even",
    "amt_merchant_diff", 
    "amt_category_diff", 
    "amt_gender_diff", 
    "amt_state_diff", 
    "amt_city_diff",
).show()

## City population

### Raw data
- high positive skewness
- both fraud nad non-fraud distributions look similair
- Ccity_pop possibly won't be good single feature
- but maybe it will interact with other

In [ ]:
show_histograms(train_df, "city_pop")

### Transform distribution

In [ ]:
show_histograms(train_df, "city_pop", 0.3)

## Gender

In [ ]:
train_df.select("gender").distinct().show()

In [ ]:
train_df = train_df.withColumn(
    "is_male",
    spark_func.when(train_df["gender"] == "M", 1).otherwise(0)
)

## Geo Location

In [ ]:
graph_data = (
    train_df
    .orderBy(spark_func.rand())
    .limit(10**4)
    .select("lat", "long", "merch_lat", "merch_long", "is_fraud")
    .toPandas()
)

In [ ]:
graph_data["lat"] = graph_data["lat"].astype(float)
graph_data["long"] = graph_data["long"].astype(float)
graph_data["merch_lat"] = graph_data["merch_lat"].astype(float)
graph_data["merch_long"] = graph_data["merch_long"].astype(float)
graph_data["is_fraud"] = graph_data["is_fraud"].astype(int)

In [ ]:
def map_figure(graph_data, col_long, col_lat, title):
    fig = go.Figure()
    
    current_df = graph_data[graph_data["is_fraud"] == 0]
    fig.add_trace(
        go.Scattergeo(
            name="Fraud = False",
            lon=current_df[col_long],
            lat=current_df[col_lat],
            mode="markers",
            marker=dict(
                size=5,
                color="green"
            ),
            opacity=0.3
        )
    )
    fig.add_trace(
        go.Scattergeo(
            name="Mean geo location: Fraud = False",
            lon=[current_df[col_long].median()],
            lat=[current_df[col_lat].median()],
            mode="markers",
            marker=dict(
                size=10,
                color="green",
                symbol="square",
                line=dict(
                    color="black",
                    width=2
                )
            )
        )
    )
    
    current_df = graph_data[graph_data["is_fraud"] == 1]
    fig.add_trace(
        go.Scattergeo(
            name="Fraud = True",
            lon=current_df[col_long],
            lat=current_df[col_lat],
            mode="markers",
            marker=dict(
                size=5,
                color="red"
            ),
            opacity=0.3
        )
    )
    fig.add_trace(
        go.Scattergeo(
            name="Mean geo location: Fraud = True",
            lon=[current_df[col_long].median()],
            lat=[current_df[col_lat].median()],
            mode="markers",
            marker=dict(
                size=10,
                color="red",
                symbol="square",
                line=dict(
                    color="black",
                    width=2
                )
            )
        )
    )
    
    fig.update_layout(
        width=1000,
        title=title
    )
    
    fig.show()

In [ ]:
map_figure(graph_data, "long", "lat", "Card holder geolocation")

In [ ]:
map_figure(graph_data, "merch_long", "merch_lat", "Merchant geolocation")